__This Notebook is just a draft if one might want to use VDSA dataset as a temporal dataset. Indeed, some years are missing (the dataset ended in 2011 and could not be linked to all years from our raw dataset). Thus one could try to extrapolate years values to next years which is nevertheless very difficult because of refresh rate (which is yearly based). Thus an average over years is used to incorporate VDSA dataset for the moment__

# Import librairies

In [228]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import numpy as np
from random import choice

# Load a dataset

In [232]:
# Load a dataset for a test
df = pd.read_excel("../dataset vdsa/Assam/dt_fert_consumption_a_web.xlsx")
df.fillna(0, inplace=True)

In [241]:
models = {}
fertilizers = ["N_TC", "P_TC", "K_TC", "NPK_TC"]
states_problems = []
for distname in df["DISTNAME"].unique():
    # For every district
    error = False
    #     print(distname)
    models[distname] = {}
    # Get the data for the district
    inter_df = df[df["DISTNAME"] == distname].reset_index()
    series = {}
    for fert in fertilizers:

        # print(pd.Series(df[df["DISTNAME"]==distname][fert]))
        # print(pd.Series(df[df["DISTNAME"]==distname]["YEAR"]))

        # Get series with fertilizer and year
        s = inter_df[["YEAR", fert]]
        # Reindex with the years
        s = s.set_index("YEAR")
        if len(s) <= 10:
            # if not enough yyears
            states_problems.append([distname, fert])
            # Take random district to fill it (could be improved by taking a random district of the same state)
            inter_df = df[
                df["DISTNAME"]
                == choice(
                    [
                        elem
                        for elem in df["DISTNAME"].unique()
                        if len(df[df["DISTNAME"] == elem]) >= 10
                    ]
                )
            ].reset_index()
            s = inter_df[["YEAR", fert]]
            s = s.set_index("YEAR")
        # Get period range for the years range from the data
        s.index = pd.period_range(start=s.index[0], periods=len(s.index), freq="A")
        # s.index=[datetime.strptime(str(elem), '%Y') for elem in s.index]

        # Define ARIMA MODEL
        model = ARIMA(s, order=(5, 1, 0), freq="A", enforce_stationarity=False)
        model_fit = model.fit()
        # print(model_fit.summary())
        # Keep track of models
        models[distname][fert] = model
        predictions = []

        # Predict the next years
        for t in range(9):
            model = ARIMA(s, freq="A", order=(5, 1, 0), enforce_stationarity=False)
            model_fit = model.fit()
            output = model_fit.forecast()
            yhat = output[0]
            predictions.append(yhat)
            s.at[str(2011 + t)] = yhat

        s["YEAR"] = s.index
        s.reset_index(drop=True, inplace=True)
        series["YEAR"] = s["YEAR"]
        s.drop(columns=["YEAR"], inplace=True)
        series[fert] = s.squeeze()

        # Add new years to the dataframe
        df2 = pd.DataFrame(
            inter_df[["STCODE", "STNAME", "DIST", "DISTNAME"]]
            .iloc[0:1]
            .loc[
                inter_df[["STCODE", "STNAME", "DIST", "DISTNAME"]]
                .iloc[0:1]
                .index.repeat(len(series["YEAR"]))
            ]
        ).reset_index(drop=True)
        df = df[df["DISTNAME"] != distname]
        df = df.append(
            pd.concat([pd.DataFrame(series), df2], axis=1), ignore_index=True
        )

    # break

/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.8/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimiz